Let's start here! If you can directly link to an image relevant to your notebook, such as [canonical logos](https://github.com/numpy/numpy/blob/main/doc/source/_static/numpylogo.svg), do so here at the top of your notebook. You can do this with Markdown syntax,

> `![<image title>](http://link.com/to/image.png "image alt text")`

or edit this cell to see raw HTML `img` demonstration. This is preferred if you need to shrink your embedded image. **Either way be sure to include `alt` text for any embedded images to make your content more accessible.**

<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>

# Accessing Argo Data

---

## Overview

Building upon previous notebook, [Introduction to Argo](notebooks/argo-introduction.ipynb), we next explore how to access Argo data in a few different ways. 

1. Data formats for Argo profiles
2. Downloading [monthly snapshots](http://www.argodatamgt.org/Access-to-data/Argo-DOI-Digital-Object-Identifier) using Argo DOI's
3. Using [Argovis](https://argovis.colorado.edu/argo) for API-based queries 
4. Using the [GO-BGC Toolbox](https://github.com/go-bgc/workshop-python)
5. Using [Argopy](https://argopy.readthedocs.io/en/latest/user-guide/fetching-argo-data/index.html), a dedicated Python package

After going through this notebook, you will be able to retrieve Argo data of interest within a certain time frame, geographical location, or by platform identifier. There are many ways of working with Argo data which are not described here. 
Further information on Argo access can be found on the [Argo website](https://argo.ucsd.edu/data/).

## Prerequisites

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Numpy](https://numpy.org/learn/) | Necessary | |
| [Intro to NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Necessary | Familiarity with metadata structure |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray.html) | Necessary | |

- **Time to learn**: 20 min


---

## Imports
Begin your body of content with another `---` divider before continuing into this section, then remove this body text and populate the following code cell with all necessary Python imports **up-front**:

In [1]:
# Import packages
import sys
import os
import numpy as np
import pandas as pd
import scipy
import xarray as xr
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from argovisHelpers import helpers as avh

## Snapshots

In [2]:
# # Base filepath. Need for Argo GDAC function.
root = '/Users/sangminsong/Library/CloudStorage/OneDrive-UW/Code/2024_Pythia/'
profile_dir = root + 'SOCCOM_GO-BGC_LoResQC_LIAR_28Aug2023_netcdf/'

In [3]:
DSdict = {}
for filename in os.listdir(profile_dir):
    if filename.endswith(".nc"):
        fp = profile_dir + filename
        single_dataset = xr.open_dataset(fp, decode_times=False)
        DSdict[filename[0:7]] = single_dataset
# DSdict['5906030']

In [4]:
DSdict['5906007']


<xarray.Dataset> Size: 2MB
Dimensions:              (N_PROF: 117, N_LEVELS: 69, NPARAMETER: 42)
Dimensions without coordinates: N_PROF, N_LEVELS, NPARAMETER
Data variables: (12/60)
    Cruise               |S11 11B ...
    Station              (N_PROF) int32 468B ...
    Lon                  (N_PROF) float64 936B ...
    Lat                  (N_PROF) float64 936B ...
    Lat_QF               (N_PROF) |S1 117B ...
    Lat_QFA              (N_PROF) float64 936B ...
    ...                   ...
    Type                 |S1 1B ...
    mon_day_yr           (N_PROF) |S10 1kB ...
    hh_mm                (N_PROF) |S5 585B ...
    Parameters           (NPARAMETER) |S19 798B ...
    JULD                 (N_PROF) float64 936B ...
    REFERENCE_DATE_TIME  object 8B ...
Attributes:
    Comments:  \n//0\n//<Encoding>UTF-8</Encoding>\n//File updated on 08/26/2...

In [11]:
profile_dir

'/Users/sangminsong/Library/CloudStorage/OneDrive-UW/Code/2024_Pythia/SOCCOM_GO-BGC_LoResQC_LIAR_28Aug2023_netcdf/'

In [14]:
# xr.open_mfdataset("/Users/sangminsong/Library/CloudStorage/OneDrive-UW/Code/2024_Pythia/SOCCOM_GO-BGC_LoResQC_LIAR_28Aug2023_netcdf/*.nc")

ValueError: unrecognized chunk manager dask - must be one of: []

## Argovis

Argovis provides an API that allows us to interact with Argo data while only downloading the exact subsets of data needed for analysis. 

### Getting started with `argovisHelpers`

We will be using the Argovis capabilities

From the Argovis tutorial: 
> In order to allocate Argovis's limited computing resources fairly, users are encouraged to register and request a free API key. This works like a password that identifies your requests to Argovis. To do so:
>
> - Visit [https://argovis-keygen.colorado.edu/](https://argovis-keygen.colorado.edu/)
> - Fill out the form under _New Account Registration_
> - An API key will be emailed to you shortly.
>
> Treat this API key like a password - don't share it or leave it anywhere public. If you ever forget it or accidentally reveal it to a third party, see the same website above to change or deactivate your token.
>
> Put your API key in the quotes in the variable below before moving on:

In [5]:
API_ROOT='https://argovis-api.colorado.edu/'
API_KEY='de6ee72a54bc5ca29dee5c801cab13fa4a354985'

We set up our query parameters using a Dictionary before requesting data: 

In [9]:
argoSearch = {
    'startDate': '2013-05-01T00:00:00Z',
    'endDate': '2023-05-01T00:00:00Z',
    'center': '-22.5,0',
    'radius': 100
}

argoProfiles = avh.query('argo', options=argoSearch, apikey=API_KEY, apiroot=API_ROOT)
argoProfiles[0]

{'_id': '1901820_256',
 'geolocation': {'type': 'Point', 'coordinates': [-22.75594, -0.2218]},
 'basin': 1,
 'timestamp': '2023-04-09T18:34:30.001Z',
 'date_updated_argovis': '2023-07-14T10:44:14.125Z',
 'source': [{'source': ['argo_core'],
   'url': 'ftp://ftp.ifremer.fr/ifremer/argo/dac/aoml/1901820/profiles/R1901820_256.nc',
   'date_updated': '2023-07-13T22:33:14.000Z'}],
 'cycle_number': 256,
 'geolocation_argoqc': 1,
 'profile_direction': 'A',
 'timestamp_argoqc': 1,
 'vertical_sampling_scheme': 'Primary sampling: averaged [nominal 2 dbar binned data sampled at 0.5 Hz from a SBE41CP]',
 'data_info': [['pressure',
   'pressure_argoqc',
   'salinity',
   'salinity_argoqc',
   'temperature',
   'temperature_argoqc'],
  ['units', 'data_keys_mode'],
  [['decibar', 'A'],
   [None, None],
   ['psu', 'A'],
   [None, None],
   ['degree_Celsius', 'A'],
   [None, None]]],
 'metadata': ['1901820_m0']}

Note that the first object in argoProfiles is a single vertical Argo "profile". 
The first 7 digits of `argoProfiles[0]['_id']` refer to a float's WMO unique identification number. 
The last three digits are the profile number. 

In the above example, we are looking at data from the 256th profile from float WMO #1901820.

In [13]:
argoProfiles[0]['_id']

'1901820_256'

We can also request metadata using the argo/meta 

In [15]:
metaOptions = {
    'id': argoProfiles[0]['metadata'][0]
}

argoMeta = avh.query('argo/meta', options=metaOptions, apikey=API_KEY, apiroot=API_ROOT)
argoMeta

[{'_id': '1901820_m0',
  'data_type': 'oceanicProfile',
  'data_center': 'AO',
  'instrument': 'profiling_float',
  'pi_name': ['BRECK OWENS', ' STEVEN JAYNE', ' P.E. ROBBINS'],
  'platform': '1901820',
  'platform_type': 'S2A',
  'fleetmonitoring': 'https://fleetmonitoring.euro-argo.eu/float/1901820',
  'oceanops': 'https://www.ocean-ops.org/board/wa/Platform?ref=1901820',
  'positioning_system': 'GPS',
  'wmo_inst_type': '854'}]

In [16]:
platformSearch = {
    'platform': argoMeta[0]['platform']
}

platformProfiles = avh.query('argo', options=platformSearch, apikey=API_KEY, apiroot=API_ROOT)
print(len(platformProfiles))

301


### Making `data` queries

We can query float profiles within these bounds: 

In [21]:
dataQuery = {
    'startDate': '2017-08-01T00:00:00Z',
    'endDate': '2017-09-01T00:00:00Z',
    'polygon': [[-150,-30],[-155,-30],[-155,-35],[-150,-35],[-150,-30]],
    'data': 'doxy'
}

profiles = avh.query('argo', options=dataQuery, apikey=API_KEY, apiroot=API_ROOT)

In [22]:
inflated_data = avh.data_inflate(profiles[0])
inflated_data[0:10]

[{'doxy': 235.335724, 'pressure': 7.6},
 {'doxy': 235.327026, 'pressure': 13.07},
 {'doxy': 235.418045, 'pressure': 17.720001},
 {'doxy': 235.212158, 'pressure': 22.02},
 {'doxy': 235.242828, 'pressure': 26.68},
 {'doxy': 235.235306, 'pressure': 31.320002},
 {'doxy': 235.273743, 'pressure': 36.709999},
 {'doxy': 235.165115, 'pressure': 41.73},
 {'doxy': 235.16153, 'pressure': 48.260002},
 {'doxy': 235.032471, 'pressure': 54.619999}]

### Subsection to the second section

#### a quick demonstration

##### of further and further

###### header levels

as well $m = a * t / h$ text! Similarly, you have access to other $\LaTeX$ equation [**functionality**](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html) via MathJax (demo below from link),

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

Check out [**any number of helpful Markdown resources**](https://www.markdownguide.org/basic-syntax/) for further customizing your notebooks and the [**Jupyter docs**](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html) for Jupyter-specific formatting information. Don't hesitate to ask questions if you have problems getting it to look *just right*.

## Last Section

If you're comfortable, and as we briefly used for our embedded logo up top, you can embed raw html into Jupyter Markdown cells (edit to see):

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    Your relevant information here!
</div>

Feel free to copy this around and edit or play around with yourself. Some other `admonitions` you can put in:

<div class="admonition alert alert-success">
    <p class="admonition-title" style="font-weight:bold">Success</p>
    We got this done after all!
</div>

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    Be careful!
</div>

<div class="admonition alert alert-danger">
    <p class="admonition-title" style="font-weight:bold">Danger</p>
    Scary stuff be here.
</div>

We also suggest checking out Jupyter Book's [brief demonstration](https://jupyterbook.org/content/metadata.html#jupyter-cell-tags) on adding cell tags to your cells in Jupyter Notebook, Lab, or manually. Using these cell tags can allow you to [customize](https://jupyterbook.org/interactive/hiding.html) how your code content is displayed and even [demonstrate errors](https://jupyterbook.org/content/execute.html#dealing-with-code-that-raises-errors) without altogether crashing our loyal army of machines!

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!